# AC Notes
## Files
### 20200303_ForestPotential_Samples.csv
- system:index
- CHELSA_Annual_Mean_Temperature
- CHELSA_Annual_Precipitation
- CHELSA_Mean_Temperature_of_Warmest_Quarter
- CHELSA_Precipitation_Seasonality
- CHELSA_Precipitation_of_Driest_Quarter
- EarthEnvTopoMed_Eastness
- EarthEnvTopoMed_Elevation
- EarthEnvTopoMed_Northness
- Lat
- Long
- Resolve_Biome
- SG_Depth_to_bedrock
- SG_Sand_Content_000cm
- SG_Sand_Content_005cm
- shrubcover
- treecover
- .geo

### 20200303_ForestPotential_PointSampling
No idea - Javascript file


In [1]:
# Import the necessary modules for both Earth Engine work as well as map visualization within a Jupyter notebook
import ee as ee
ee.Initialize()
import folium
import geehydro #AC This isnt used anywhere
import time
import datetime
import numpy as np
import subprocess

### users/devinrouth/Resolve_Biomes_30ArcSec dataset
Has 14 biomes:
https://developers.google.com/earth-engine/datasets/catalog/RESOLVE_ECOREGIONS_2017

### users/devinrouth/Future_BioClim_Ensembles/rcp* datasets
These are WorldClim data with added variables for ecology.  
BIOCLIM variables https://www.edenextdata.com/?q=content/bioclim-worldclim-bioclimatic-variables-2000-20-50-80  
The years are 2030,2050,2070,2080  
The RCPS are 4.5 and 8.5  
Have 19 variables:  
Annual_Mean_Temperature: 29.585416666666667  
Mean_Diurnal_Range: 9.506770833333333  
Isothermality: 68.97395833333333  
Temperature_Seasonality: 119.45416666666667  
Max_Temperature_of_Warmest_Month: 37.36770833333333  
Min_Temperature_of_Coldest_Month: 23.684375  
Temperature_Annual_Range: 13.683333333333334  
Mean_Temperature_of_Wettest_Quarter: 28.03333333333333  
Mean_Temperature_of_Driest_Quarter: 30.833854166666665  
Mean_Temperature_of_Warmest_Quarter: 31.144791666666666  
Mean_Temperature_of_Coldest_Quarter: 27.9859375  
Annual_Precipitation: 1932.15625  
Precipitation_of_Wettest_Month: 277.9375  
Precipitation_of_Driest_Month: 15.052083333333334  
Precipitation_Seasonality: 58.651041666666664  
Precipitation_of_Wettest_Quarter: 782.515625  
Precipitation_of_Driest_Quarter: 77.85416666666667  
Precipitation_of_Warmest_Quarter: 140.3125  
Precipitation_of_Coldest_Quarter: 780.9010416666666  

### "users/devinrouth/ETH_Composites/CrowtherLab_Composite_30ArcSec" dataset
This has 302 bands!! Including:  
Aridity index  
depth to water table  
EVI  
Cloud cover statistics  
Texture measurements including Shannon and Simpson  
Aspect, elevation, roughness, slope  
FPAR  
Above ground biomass  
Hansen Forest cover  
Human footprint  
Landcover class  
Mycorrhizae  
NDVI ?  
Nematodes  
Lots of soil properties  
Tree density  
Chelsa Climate data http://chelsa-climate.org/  

In [2]:
# Load the datasets of interest
resolveBiomes = ee.Image("users/devinrouth/Resolve_Biomes_30ArcSec")
rcp45_2080s_Mean = ee.Image("users/devinrouth/Future_BioClim_Ensembles/rcp45_2080s_Mean")
rcp45_2030s_Mean = ee.Image("users/devinrouth/Future_BioClim_Ensembles/rcp45_2030s_Mean")
rcp45_2050s_Mean = ee.Image("users/devinrouth/Future_BioClim_Ensembles/rcp45_2050s_Mean")
rcp45_2070s_Mean = ee.Image("users/devinrouth/Future_BioClim_Ensembles/rcp45_2070s_Mean")
rcp85_2030s_Mean = ee.Image("users/devinrouth/Future_BioClim_Ensembles/rcp85_2030s_Mean")
rcp85_2050s_Mean = ee.Image("users/devinrouth/Future_BioClim_Ensembles/rcp85_2050s_Mean")
rcp85_2070s_Mean = ee.Image("users/devinrouth/Future_BioClim_Ensembles/rcp85_2070s_Mean")
rcp85_2080s_Mean = ee.Image("users/devinrouth/Future_BioClim_Ensembles/rcp85_2080s_Mean")
composite = ee.Image("users/devinrouth/ETH_Composites/CrowtherLab_Composite_30ArcSec")

# Create an unbounded geometry for later use
# unboundedGeo = ee.Geometry.Rectangle([-180, -90, 180, 90], "EPSG:4326", False)
unboundedGeo = ee.Geometry.Polygon([-180, 88, 0, 88, 180, 88, 180, -88, 0, -88, -180, -88], None, False);

# !! Change testGeo when running full pipeline
testGeo = ee.Geometry.Polygon(
        [[[8.453552577781124, 47.44586460369053],
          [8.453552577781124, 47.2896051444241],
          [8.728210780906124, 47.2896051444241],
          [8.728210780906124, 47.44586460369053]]], None, False);

# Decide what geometry to use for exports (i.e., determining the extent of the maps)
exportingGeometry = unboundedGeo

# Create a list of biome number designations for later use
biomeNumberList = [1,2,3,4,5,6,7,8,9,10,11,12,13,14]

# Input the username folder where all of the images and collections will be saved
# usernameFolderString = 'devinrouth_backup'
usernameFolderString = 'acottam'

# Input the name of the project folder wherein all (top-level) items will be saved
# !! This folder must be created in order for the script to run
projectFolder = 'ETH_Biome_Future_Predictions'

# Input the name of the folder that will hold the bootstrapped samples
bootstrapCollFolder = 'Bootstrap_Samples'

# Select how many points per biome you'd like to sample
pointsPerBiome = 2500

# Select a tileScale to use across sampling calls
# AC The default is 1 and this may be why it takes so long?
tileScaleToUse = 16

# Input the normal wait time (in seconds) for "wait and break" cells
normalWaitTime = 60

# Input the long wait time (in seconds) for "wait and break" cells
longWaitTime = 600

In [3]:
# Write a list of random numbers to serve as the seed for the bootstrap collections; the number of seeds will determine the number of bootstraps
# !! This list determines how many bootstrap samples will be created / modeled
#seedList = list(range(1,1001))
seedList = list(range(1,11))
print(seedList)
print('\n\n\n')

# Choose how many collections/images to use/make at a time when bootstrapping
n = 10
seedChunkListToMap = [seedList[i:i + n] for i in range(0, len(seedList), n)]
print(seedChunkListToMap)
print('\n\n\n')


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]




[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]]






In [4]:
# Create bash variables in order to create/check/delete Earth Engine Assets
bashFunction = 'earthengine'
arglist_CreateCollection = ['--no-use_cloud_api','create','collection']
arglist_CreateFolder = ['--no-use_cloud_api','create','folder']
arglist_Detect = ['--no-use_cloud_api','asset','info']
arglist_Delete = ['--no-use_cloud_api','rm','-r']
stringsOfInterest = ['Asset does not exist or is not accessible']
bashCommandList_Detect = [bashFunction]+arglist_Detect
bashCommandList_Delete = [bashFunction]+arglist_Delete
bashCommandList_CreateCollection = [bashFunction]+arglist_CreateCollection
bashCommandList_CreateFolder = [bashFunction]+arglist_CreateFolder


In [5]:
# Make a scaling image to ensure the scale of the CHELSA images matches the other images
scalingDictionary = ee.Dictionary(
{
'CHELSA_Annual_Mean_Temperature':0.1,
'CHELSA_Annual_Precipitation':1,
'CHELSA_Isothermality':0.1,
'CHELSA_Max_Temperature_of_Warmest_Month':0.1,
'CHELSA_Mean_Diurnal_Range':0.1,
'CHELSA_Mean_Temperature_of_Coldest_Quarter':0.1,
'CHELSA_Mean_Temperature_of_Driest_Quarter':0.1,
'CHELSA_Mean_Temperature_of_Warmest_Quarter':0.1,
'CHELSA_Mean_Temperature_of_Wettest_Quarter':0.1,
'CHELSA_Min_Temperature_of_Coldest_Month':0.1,
'CHELSA_Precipitation_Seasonality':1,
'CHELSA_Precipitation_of_Coldest_Quarter':1,
'CHELSA_Precipitation_of_Driest_Month':1,
'CHELSA_Precipitation_of_Driest_Quarter':1,
'CHELSA_Precipitation_of_Warmest_Quarter':1,
'CHELSA_Precipitation_of_Wettest_Month':1,
'CHELSA_Precipitation_of_Wettest_Quarter':1,
'CHELSA_Temperature_Annual_Range':0.1,
'CHELSA_Temperature_Seasonality':0.1
});


In [6]:
# Scale the CHELSA bands from the composite to match the future predictions on these bands
# AC This is taking the above bands and multiplying their values by a constant image with the scale factors! Nice.
chelsaMultibandImage = composite.select(scalingDictionary.keys()).multiply(scalingDictionary.toImage());


In [7]:
# Compute lists of other variables to include in the models (i.e., soil and topographic variables)
# AC topComp has the topographic variables: slope, elevation and aspect
topComp = composite.select(['EarthEnvTopoMed_Slope','EarthEnvTopoMed_Elevation','EarthEnvTopoMed_Eastness','EarthEnvTopoMed_Northness']);
# AC soilComp has the soil variables: organic carbon, pH, density, sand, water capacity, cation exchange capacity (CEC)
soilComp = composite.select(['SG_CEC_000cm','SG_SOC_Density_000cm','SG_Soil_pH_H2O_000cm','SG_Bulk_density_000cm','SG_Sand_Content_000cm','SG_H2O_Capacity_000cm']);

In [8]:
# Concatenate the two images into a single variable
totComp = chelsaMultibandImage.addBands(soilComp).addBands(topComp);
# AC so the independent variables in the model are the following:
# CHELSA climate, topographic and soil variables
# These are used to train the Random Forest classifier
covariateNames = totComp.bandNames()

In [9]:
# First, make a dictionary to translate the band names
# AC Between the CHELSA climate data dataset and the RCP scenario data (e.g. users/devinrouth/Future_BioClim_Ensembles/rcp45_2080s_Mean)
chelsaDict =ee.Dictionary({
'Annual_Mean_Temperature':'CHELSA_Annual_Mean_Temperature',
'Mean_Diurnal_Range':'CHELSA_Mean_Diurnal_Range',
'Isothermality':'CHELSA_Isothermality',
'Temperature_Seasonality':'CHELSA_Temperature_Seasonality',
'Max_Temperature_of_Warmest_Month':'CHELSA_Max_Temperature_of_Warmest_Month',
'Min_Temperature_of_Coldest_Month':'CHELSA_Min_Temperature_of_Coldest_Month',
'Temperature_Annual_Range':'CHELSA_Temperature_Annual_Range',
'Mean_Temperature_of_Wettest_Quarter':'CHELSA_Mean_Temperature_of_Wettest_Quarter',
'Mean_Temperature_of_Driest_Quarter':'CHELSA_Mean_Temperature_of_Driest_Quarter',
'Mean_Temperature_of_Warmest_Quarter':'CHELSA_Mean_Temperature_of_Warmest_Quarter',
'Mean_Temperature_of_Coldest_Quarter':'CHELSA_Mean_Temperature_of_Coldest_Quarter',
'Annual_Precipitation':'CHELSA_Annual_Precipitation',
'Precipitation_of_Wettest_Month':'CHELSA_Precipitation_of_Wettest_Month',
'Precipitation_of_Driest_Month':'CHELSA_Precipitation_of_Driest_Month',
'Precipitation_Seasonality':'CHELSA_Precipitation_Seasonality',
'Precipitation_of_Wettest_Quarter':'CHELSA_Precipitation_of_Wettest_Quarter',
'Precipitation_of_Driest_Quarter':'CHELSA_Precipitation_of_Driest_Quarter',
'Precipitation_of_Warmest_Quarter':'CHELSA_Precipitation_of_Warmest_Quarter',
'Precipitation_of_Coldest_Quarter':'CHELSA_Precipitation_of_Coldest_Quarter'
});


In [10]:
# Prepare all of the future imagery
# AC This selects the climate fields in the Bioclim data and renames them
rcp45_2030s_Mean = rcp45_2030s_Mean.select(chelsaDict.keys(),chelsaDict.values());
rcp45_2050s_Mean = rcp45_2050s_Mean.select(chelsaDict.keys(),chelsaDict.values());
rcp45_2070s_Mean = rcp45_2070s_Mean.select(chelsaDict.keys(),chelsaDict.values());
rcp45_2080s_Mean = rcp45_2080s_Mean.select(chelsaDict.keys(),chelsaDict.values());

rcp85_2030s_Mean = rcp85_2030s_Mean.select(chelsaDict.keys(),chelsaDict.values());
rcp85_2050s_Mean = rcp85_2050s_Mean.select(chelsaDict.keys(),chelsaDict.values());
rcp85_2070s_Mean = rcp85_2070s_Mean.select(chelsaDict.keys(),chelsaDict.values());
rcp85_2080s_Mean = rcp85_2080s_Mean.select(chelsaDict.keys(),chelsaDict.values());

# AC This adds the soil and topographic variables to the Bioclim data
rcp45_2030s_Mean_Soil = rcp45_2030s_Mean.addBands(soilComp).addBands(topComp);
rcp45_2050s_Mean_Soil = rcp45_2050s_Mean.addBands(soilComp).addBands(topComp);
rcp45_2070s_Mean_Soil = rcp45_2070s_Mean.addBands(soilComp).addBands(topComp);
rcp45_2080s_Mean_Soil = rcp45_2080s_Mean.addBands(soilComp).addBands(topComp);

rcp85_2030s_Mean_Soil = rcp85_2030s_Mean.addBands(soilComp).addBands(topComp);
rcp85_2050s_Mean_Soil = rcp85_2050s_Mean.addBands(soilComp).addBands(topComp);
rcp85_2070s_Mean_Soil = rcp85_2070s_Mean.addBands(soilComp).addBands(topComp);
rcp85_2080s_Mean_Soil = rcp85_2080s_Mean.addBands(soilComp).addBands(topComp);


In [11]:
# Compute the training and test samples
trainSamplesToExport = totComp.addBands(resolveBiomes.rename('ResolveBiome').int()).stratifiedSample(
    numPoints=pointsPerBiome,
    classBand='ResolveBiome',
    region=unboundedGeo,
    seed=11111,
    tileScale=tileScaleToUse,
    geometries=True
)

validateSamplesToExport = totComp.addBands(resolveBiomes.rename('ResolveBiome').int()).stratifiedSample(
    numPoints=pointsPerBiome,
    classBand='ResolveBiome',
    region=unboundedGeo,
    seed=22222,
    tileScale=tileScaleToUse,
    geometries=True
)

testSamplesToExport = totComp.addBands(resolveBiomes.rename('ResolveBiome').int()).stratifiedSample(
    numPoints=pointsPerBiome,
    classBand='ResolveBiome',
    region=unboundedGeo,
    seed=33333,
    tileScale=tileScaleToUse,
    geometries=True
)


In [12]:
# Perform the export then wait for it to finish before moving on
trainingSampleExport = ee.batch.Export.table.toAsset(
    collection=trainSamplesToExport,
    description='trainingSamples',
    assetId='users/'+usernameFolderString+'/'+projectFolder+'/trainingSamples'
);
trainingSampleExport.start()

validateSampleExport = ee.batch.Export.table.toAsset(
    collection=validateSamplesToExport,
    description='validateSamples',
    assetId='users/'+usernameFolderString+'/'+projectFolder+'/validateSamples'
);
validateSampleExport.start()

testSampleExport = ee.batch.Export.table.toAsset(
    collection=testSamplesToExport,
    description='testSamples',
    assetId='users/'+usernameFolderString+'/'+projectFolder+'/testSamples'
);
testSampleExport.start()

# Sleep to ensure the jobs have been queued
time.sleep(normalWaitTime/2)


In [13]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


Moving on...


In [14]:
# Load the collections
trainSamples = ee.FeatureCollection('users/'+usernameFolderString+'/'+projectFolder+'/trainingSamples')
validateSamples = ee.FeatureCollection('users/'+usernameFolderString+'/'+projectFolder+'/validateSamples')
testSamples = ee.FeatureCollection('users/'+usernameFolderString+'/'+projectFolder+'/testSamples')


In [15]:
# Instantiate a selection of random forest classifiers to determine the best model (using features for wrapping, so the entire process can be processed via an export task)
rf_VP2 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP2','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=2,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP3 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP3','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=3,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP4 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP4','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=4,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP5 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP5','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=5,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP6 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP6','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=6,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP7 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP7','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=7,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP8 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP8','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=8,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP9 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP9','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=9,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP10 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP10','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=10,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP12 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP12','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=12,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))

rf_VP14 = ee.Feature(ee.Geometry.Point([0,0])).set('cName','rf_VP14','c',ee.Classifier.smileRandomForest(
    numberOfTrees=100,
    variablesPerSplit=14,
    bagFraction=0.632
).setOutputMode('CLASSIFICATION').train(trainSamples,'ResolveBiome',covariateNames))



In [16]:
# Map through the trained classifiers, classify the validation samples, then compute their accuracy to compare them
classifierFC = ee.FeatureCollection([rf_VP2,
                                     rf_VP3,
                                     rf_VP4,
                                     rf_VP5,
                                     rf_VP6,
                                     rf_VP7,
                                     rf_VP8,
                                     rf_VP9,
                                     rf_VP10,
                                     rf_VP12,
                                     rf_VP14])

def outputValidationAccuracy(classiferFeature):
    return ee.Feature(classiferFeature).set('OverallAccuracy',validateSamples.classify(ee.Feature(classiferFeature).get('c'),'PredictedBiome').errorMatrix('ResolveBiome','PredictedBiome').accuracy()).select(['OverallAccuracy','cName'])

# !! Export the accuracy FC
accuracyFC = classifierFC.map(outputValidationAccuracy).sort('OverallAccuracy',False)

finalClassifierFCExport = ee.batch.Export.table.toAsset(
    collection=ee.FeatureCollection(accuracyFC),
    description='finalClassifierFC',
    assetId='users/'+usernameFolderString+'/'+projectFolder+'/finalClassifierFC'
);
finalClassifierFCExport.start()

# Sleep to ensure the job has been queued
time.sleep(normalWaitTime/2)


In [17]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


Moving on...


In [18]:
# !! Export the best model information separately
bestModelDictionary = ee.Feature(accuracyFC.first()).toDictionary()
finalValidationFCExport = ee.batch.Export.table.toAsset(
    collection=ee.FeatureCollection(ee.Feature(ee.Geometry.Point([0,0])).set(bestModelDictionary)),
    description='finalValidationFC',
    assetId='users/'+usernameFolderString+'/'+projectFolder+'/finalValidationFC'
);
finalValidationFCExport.start()

# Sleep to ensure the job has been queued
time.sleep(normalWaitTime/2)


In [19]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


Moving on...


In [20]:
# Compute the final accuracy on the test collection
bestModelName = ee.Feature(ee.FeatureCollection('users/'+usernameFolderString+'/'+projectFolder+'/finalValidationFC').first()).get('cName')
bestModel = ee.Feature(classifierFC.filterMetadata('cName','equals',bestModelName).first()).get('c')
finalAccuracy = testSamples.classify(ee.Classifier(bestModel),'PredictedBiome').errorMatrix('ResolveBiome','PredictedBiome',biomeNumberList).accuracy()
kappaAccuracy = testSamples.classify(ee.Classifier(bestModel),'PredictedBiome').errorMatrix('ResolveBiome','PredictedBiome',biomeNumberList).kappa()

# !! Export the final accuracy
finalAccuracyFCExport = ee.batch.Export.table.toAsset(
    collection=ee.FeatureCollection(ee.Feature(ee.Geometry.Point([0,0])).set('FinalAccuracy',finalAccuracy).set('KappaAccuracy',kappaAccuracy)),
    description='finalAccuracyFC',
    assetId='users/'+usernameFolderString+'/'+projectFolder+'/finalAccuracyFC'
);
finalAccuracyFCExport.start()

# Sleep to ensure the job has been queued
time.sleep(normalWaitTime/2)


In [21]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


Moving on...


In [22]:
# Return accuracy values here
finalAccuracyToPrint = ee.Feature(ee.FeatureCollection('users/'+usernameFolderString+'/'+projectFolder+'/finalAccuracyFC').first()).get('FinalAccuracy')
print('FinalAccuracy');
print(finalAccuracyToPrint.getInfo(),'\n')

kappaAccuracyToPrint = ee.Feature(ee.FeatureCollection('users/'+usernameFolderString+'/'+projectFolder+'/finalAccuracyFC').first()).get('KappaAccuracy')
print('KappaAccuracy');
print(kappaAccuracyToPrint.getInfo(),'\n')


FinalAccuracy
0.8940857142857143 

KappaAccuracy
0.8859384615384616 



In [23]:
# Show the raw confusion matrix for the test data
# !! Use a while loop here to repeat the code in case there are Computation time out errors
while True:
    try:
        confusionMatrix = testSamples.classify(ee.Classifier(bestModel),'PredictedBiome').errorMatrix('ResolveBiome','PredictedBiome',biomeNumberList).array().getInfo()
        print(confusionMatrix)
        print('\n')
    except Exception as e:
        print(e,' : ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
        time.sleep(30)
        continue
    else:
         break

for r in confusionMatrix:
    print(*r)


[[2008, 163, 45, 17, 6, 0, 97, 0, 3, 15, 0, 0, 1, 145], [73, 2151, 111, 0, 0, 0, 37, 0, 31, 15, 0, 0, 7, 75], [33, 31, 2395, 2, 7, 0, 1, 0, 0, 4, 0, 0, 1, 26], [13, 0, 8, 2149, 112, 35, 6, 55, 33, 21, 0, 54, 12, 2], [5, 0, 6, 62, 2138, 27, 0, 49, 1, 128, 7, 40, 37, 0], [0, 0, 0, 42, 56, 2233, 0, 25, 1, 0, 143, 0, 0, 0], [75, 69, 7, 4, 0, 0, 2219, 13, 59, 10, 0, 1, 33, 10], [0, 0, 0, 82, 70, 30, 3, 2205, 16, 18, 0, 39, 36, 1], [5, 13, 1, 6, 0, 0, 80, 5, 2350, 0, 0, 3, 28, 9], [17, 1, 16, 19, 125, 0, 33, 30, 0, 2191, 0, 6, 62, 0], [0, 0, 0, 6, 11, 192, 0, 0, 0, 2, 2289, 0, 0, 0], [0, 0, 0, 22, 31, 0, 0, 6, 2, 9, 0, 2396, 34, 0], [2, 50, 32, 2, 22, 0, 36, 56, 19, 54, 0, 42, 2174, 11], [59, 24, 11, 0, 0, 0, 2, 0, 5, 0, 0, 0, 4, 2395]]


2008 163 45 17 6 0 97 0 3 15 0 0 1 145
73 2151 111 0 0 0 37 0 31 15 0 0 7 75
33 31 2395 2 7 0 1 0 0 4 0 0 1 26
13 0 8 2149 112 35 6 55 33 21 0 54 12 2
5 0 6 62 2138 27 0 49 1 128 7 40 37 0
0 0 0 42 56 2233 0 25 1 0 143 0 0 0
75 69 7 4 0 0 2219 13 59 10 0 1 

In [24]:
# Show the consumers' accuracy for the test data
# !! Use a while loop here to repeat the code in case there are Computation time out errors
while True:
    try:
        consumersMatrix = testSamples.classify(ee.Classifier(bestModel),'PredictedBiome').errorMatrix('ResolveBiome','PredictedBiome',biomeNumberList).consumersAccuracy().getInfo()[0]
        print(consumersMatrix)
        print('\n')
    except Exception as e:
        print(e,' : ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
        time.sleep(30)
        continue
    else:
         break

for b in biomeNumberList:
    print('Biome ',b,': ',consumersMatrix[b-1])


[0.8768558951965065, 0.8597122302158273, 0.9099544072948328, 0.8905926232905097, 0.8293250581846393, 0.8871672626142233, 0.8826571201272871, 0.9022094926350246, 0.9325396825396826, 0.8881232265910012, 0.9384993849938499, 0.9283223556760946, 0.8950185261424455, 0.8956619296933434]


Biome  1 :  0.8768558951965065
Biome  2 :  0.8597122302158273
Biome  3 :  0.9099544072948328
Biome  4 :  0.8905926232905097
Biome  5 :  0.8293250581846393
Biome  6 :  0.8871672626142233
Biome  7 :  0.8826571201272871
Biome  8 :  0.9022094926350246
Biome  9 :  0.9325396825396826
Biome  10 :  0.8881232265910012
Biome  11 :  0.9384993849938499
Biome  12 :  0.9283223556760946
Biome  13 :  0.8950185261424455
Biome  14 :  0.8956619296933434


In [25]:
# Print the producers' accuracy values
# !! Use a while loop here to repeat the code in case there are Computation time out errors
while True:
    try:
        producersMatrix = testSamples.classify(ee.Classifier(bestModel),'PredictedBiome').errorMatrix('ResolveBiome','PredictedBiome',biomeNumberList).producersAccuracy().getInfo()
        print(producersMatrix)
        print('\n')
    except Exception as e:
        print(e,' : ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
        time.sleep(30)
        continue
    else:
         break

for b in biomeNumberList:
    print('Biome ',b,': ',producersMatrix[b-1][0])


[[0.8032], [0.8604], [0.958], [0.8596], [0.8552], [0.8932], [0.8876], [0.882], [0.94], [0.8764], [0.9156], [0.9584], [0.8696], [0.958]]


Biome  1 :  0.8032
Biome  2 :  0.8604
Biome  3 :  0.958
Biome  4 :  0.8596
Biome  5 :  0.8552
Biome  6 :  0.8932
Biome  7 :  0.8876
Biome  8 :  0.882
Biome  9 :  0.94
Biome  10 :  0.8764
Biome  11 :  0.9156
Biome  12 :  0.9584
Biome  13 :  0.8696
Biome  14 :  0.958


# Bootstrapping

In [26]:
# Create a folder to house the bootstrapped feature collection

# Turn the folder string into an assetID and perform the deletion
assetIDToCreate_Folder = 'users/'+usernameFolderString+'/'+projectFolder+'/'+bootstrapCollFolder
print(assetIDToCreate_Folder,'being created...')

# Create the image collection before classifying each of the bootstrap images
subprocess.run(bashCommandList_CreateFolder+[assetIDToCreate_Folder])
while any(x in subprocess.run(bashCommandList_Detect+[assetIDToCreate_Folder], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for asset to be created...')
    time.sleep(normalWaitTime)
print('Asset created!')


# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


users/acottam/ETH_Biome_Future_Predictions/Bootstrap_Samples being created...
Asset created!


In [27]:
# Write a function load each of the collections in, chunk by chunk, and use them to predict a composite
# Create the bootstrap collections for training
for seed in seedList:
    assetIDString = str('BootstrapSamples_')+str(seed).zfill(4)
    bootstrapSamples = totComp.addBands(resolveBiomes.rename('ResolveBiome').int()).stratifiedSample(
    numPoints=pointsPerBiome,
    classBand='ResolveBiome',
    region=unboundedGeo,
    seed=seed,
    tileScale=tileScaleToUse,
    geometries=True
    )
    
    bootstrapSampleExport = ee.batch.Export.table.toAsset(
        collection=bootstrapSamples,
        description=assetIDString,
        assetId='users/'+usernameFolderString+'/'+projectFolder+'/'+bootstrapCollFolder+'/'+assetIDString
    );
    bootstrapSampleExport.start()
    

In [28]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(longWaitTime)
print('Moving on...')


You have jobs running!  2020-04-05 11:36:47
Moving on...


In [29]:
# Convert the categorical image to a multi-band indicator variable style image
# AC This is the WWF Global 200 - https://en.wikipedia.org/wiki/Global_200
resolveBiomesDict = {
1: 'Tropical and subtropical moist broadleaf forests',
2: 'Tropical and subtropical dry broadleaf forests',
3: 'Tropical and subtropical coniferous forests',
4: 'Temperate broadleaf and mixed forests',
5: 'Temperate conifer forests',
6: 'Boreal forests or taiga',
7: 'Tropical and subtropical grasslands, savannas, and shrublands',
8: 'Temperate grasslands, savannas, and shrublands',
9: 'Flooded Grasslands and Savannas',
10: 'Montane grasslands and shrublands',
11: 'Tundra',
12: 'Mediterranean forests, woodlands, and scrub',
13: 'Deserts and xeric shrublands',
14: 'Mangroves'
};

# Instantiate each of the desired function arguments before the scope of the function so as to have only a single argument to the function
classBandToMap = 'Resolve_Biome';
classDictToMap = resolveBiomesDict;
imageNameForBands = 'ResolveBiome';
def categoricalToIndicatorWithoutRefToApply(inputCatImage):
    def makeBooleanRasterFromValue(i):
        return ee.Image(inputCatImage).select(classBandToMap).eq(ee.Image(ee.Number.parse(i))).rename(ee.String(imageNameForBands).cat(ee.String('_')).cat(ee.Number.parse(i).format('%02d')));
    # AC Make a boolean image for each biome and add it to an image list
    catClassImageList = ee.Dictionary(classDictToMap).keys().map(makeBooleanRasterFromValue);
    def makeRasterFromImageList(i,b):
        return ee.Image(b).addBands(i)
    #AC Converts the image list to a single image with bands ResolveBiome_01, ResolveBiome_02 etc
    unsortedBandImage = ee.Image(catClassImageList.iterate(makeRasterFromImageList,ee.Image()))
    return unsortedBandImage.select(unsortedBandImage.bandNames().remove('constant').sort());



In [30]:
# Define a function to make the bootstrap indicator images and save them to a recipient image collection
def makeBootstrapIC(imageToClassify,parentColl,listOfLists):

    for seeds in listOfLists:
        seedChunkList = seeds
        indexString = str(listOfLists.index(seeds)).zfill(2)

        fcAssetIDList = []
        for seed in seedChunkList:
            assetIDString = str('BootstrapSamples_')+str(seed).zfill(4)
            # AC load the bootstrap training samples
            assetId='users/'+usernameFolderString+'/'+projectFolder+'/'+bootstrapCollFolder+'/'+assetIDString
            fcAssetIDList.append(assetId)
            def returnFCFromString(string):
                return ee.FeatureCollection(string)
            # AC creates an image collection with the image to classify with a property for the bootstrap samples
            imageCollWithTrainingColls = ee.ImageCollection(ee.List(list(map(returnFCFromString,fcAssetIDList)))
                                                            .map(lambda s: imageToClassify.set('TrainingCollection',ee.FeatureCollection(s),
                                                                                               'TrainingString',s)))
        def classifyImage(imageWithColl):
            trainingCollToUse = ee.FeatureCollection(imageWithColl.get('TrainingCollection'))
            trainedModel = ee.Classifier(bestModel).setOutputMode('CLASSIFICATION').train(trainingCollToUse,'ResolveBiome',covariateNames)
            classifiedImage = imageWithColl.classify(trainedModel,'PredictedBiome').rename('Resolve_Biome')
            return categoricalToIndicatorWithoutRefToApply(classifiedImage.copyProperties(imageWithColl,['TrainingString']))


        classifiedImageCollSum = ee.ImageCollection(imageCollWithTrainingColls.map(classifyImage)).sum()
        ImageExport = ee.batch.Export.image.toAsset(
            image=classifiedImageCollSum,
            description=parentColl+'_'+'BootstrapImage_'+indexString,
            assetId='users/'+usernameFolderString+'/'+projectFolder+'/'+parentColl+'/BootstrapImage_'+indexString,
            crs='EPSG:4326',
            crsTransform='[0.008333333333333333,0,-180,0,-0.008333333333333333,90]',
            region=exportingGeometry.getInfo()['coordinates'],
            maxPixels=int(1e13)
        );
        ImageExport.start()



# Bootstrap each of the time periods, sum the collection, then delete the image collection (to make room in the account asset storage)

In [31]:
# usernameFolderString = 'devinrouth_backup'
usernameFolderString = 'acottam'
projectFolder = 'ETH_Biome_Future_Predictions'

In [32]:
# Create bash variables in order to create/check/delete Earth Engine Assets
bashFunction = 'earthengine'
arglist_CreateCollection = ['--no-use_cloud_api','create','collection']
arglist_CreateFolder = ['--no-use_cloud_api','create','folder']
arglist_Detect = ['--no-use_cloud_api','asset','info']
arglist_Delete = ['--no-use_cloud_api','rm','-r']
stringsOfInterest = ['Asset does not exist or is not accessible']
bashCommandList_Detect = [bashFunction]+arglist_Detect
bashCommandList_Delete = [bashFunction]+arglist_Delete
bashCommandList_CreateCollection = [bashFunction]+arglist_CreateCollection
bashCommandList_CreateFolder = [bashFunction]+arglist_CreateFolder


## rcp45_2030s

In [33]:
# rcp45_2030s Image Collection creation
rcp45_2030s_Coll = 'rcp45_2030s_Coll';

# Turn the coll string into an assetID and perform the deletion
currentCollToCreate = rcp45_2030s_Coll
assetIDToCreate = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToCreate
print(assetIDToCreate,'being created...')

# Create the image collection before classifying each of the bootstrap images
subprocess.run(bashCommandList_CreateCollection+[assetIDToCreate])
while any(x in subprocess.run(bashCommandList_Detect+[assetIDToCreate], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for asset to be created...')
    time.sleep(normalWaitTime)
print('Asset created!')

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


users/acottam/ETH_Biome_Future_Predictions/rcp45_2030s_Coll being created...
Asset created!


In [34]:
# Make the bootstrap images inside of the collection
# AC params to makeBootstrapIC are: image to classify, parent collection, list of lists
rcp45_2030s_Classified = makeBootstrapIC(rcp45_2030s_Mean_Soil,rcp45_2030s_Coll,seedChunkListToMap);

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [35]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(longWaitTime)
print('Moving on...')


You have jobs running!  2020-04-05 11:48:00
You have jobs running!  2020-04-05 11:58:01
You have jobs running!  2020-04-05 12:08:02
You have jobs running!  2020-04-05 12:18:03
You have jobs running!  2020-04-05 12:28:04
You have jobs running!  2020-04-05 12:38:05
You have jobs running!  2020-04-05 12:48:06
You have jobs running!  2020-04-05 12:58:07
You have jobs running!  2020-04-05 13:08:09
You have jobs running!  2020-04-05 13:18:10
You have jobs running!  2020-04-05 13:28:11
You have jobs running!  2020-04-05 13:38:12
Moving on...


In [36]:
# Make a function to load in an image collection (by asset string name) then sum it an export the resultant image
def sumImageCollection(collStringName):
    pathOfImageCollToSum = 'users/'+usernameFolderString+'/'+projectFolder+'/'+collStringName;
    outputPathOfSummedImage = 'users/'+usernameFolderString+'/'+projectFolder+'/'+collStringName+'_SummedImage';
    summedImageToExport = ee.ImageCollection(pathOfImageCollToSum).sum()
    
    summedImageExportTask = ee.batch.Export.image.toAsset(
        image=summedImageToExport,
        description=collStringName+'_SummedImage',
        assetId=outputPathOfSummedImage,
        crs='EPSG:4326',
        crsTransform='[0.008333333333333333,0,-180,0,-0.008333333333333333,90]',
        region=exportingGeometry.getInfo()['coordinates'],
        maxPixels=int(1e13)
    );
    summedImageExportTask.start()

In [37]:
# Sum each of the image collections to get a finalized map
sumImageCollection(rcp45_2030s_Coll)


In [38]:
# !! Break and wait
# Wait while all queued tasks finish
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


You have jobs running!  2020-04-05 13:48:16
You have jobs running!  2020-04-05 13:49:17
You have jobs running!  2020-04-05 13:50:17
You have jobs running!  2020-04-05 13:51:18
You have jobs running!  2020-04-05 13:52:19
You have jobs running!  2020-04-05 13:53:19
You have jobs running!  2020-04-05 13:54:19
You have jobs running!  2020-04-05 13:55:20
You have jobs running!  2020-04-05 13:56:20
You have jobs running!  2020-04-05 13:57:21
You have jobs running!  2020-04-05 13:58:21
You have jobs running!  2020-04-05 13:59:22
You have jobs running!  2020-04-05 14:00:23
You have jobs running!  2020-04-05 14:01:23
You have jobs running!  2020-04-05 14:02:23
You have jobs running!  2020-04-05 14:03:24
You have jobs running!  2020-04-05 14:04:24
You have jobs running!  2020-04-05 14:05:25
You have jobs running!  2020-04-05 14:06:25
You have jobs running!  2020-04-05 14:07:26
You have jobs running!  2020-04-05 14:08:26
Moving on...


In [39]:
# rcp45_2030s_Coll
currentCollToDelete = rcp45_2030s_Coll

# Turn the coll string into an assetID and perform the deletion
assetIDToDelete = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToDelete
print(assetIDToDelete,'being deleted')

# Delete the image collection after summing the 
subprocess.run(bashCommandList_Delete+[assetIDToDelete])
while not all(x in subprocess.run(bashCommandList_Detect+[assetIDToDelete], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for the asset to delete...')
    time.sleep(5)
print('Collection is deleted!')


users/acottam/ETH_Biome_Future_Predictions/rcp45_2030s_Coll being deleted
Collection is deleted!


## rcp45_2050s

In [40]:
# rcp45_2050s Image Collection creation
rcp45_2050s_Coll = 'rcp45_2050s_Coll';

# Turn the coll string into an assetID and perform the deletion
currentCollToCreate = rcp45_2050s_Coll
assetIDToCreate = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToCreate
print(assetIDToCreate,'being created...')

# Create the image collection before classifying each of the bootstrap images
subprocess.run(bashCommandList_CreateCollection+[assetIDToCreate])
while any(x in subprocess.run(bashCommandList_Detect+[assetIDToCreate], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for asset to be created...')
    time.sleep(normalWaitTime)
print('Asset created!')

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


users/acottam/ETH_Biome_Future_Predictions/rcp45_2050s_Coll being created...
Asset created!


In [41]:
# Make the bootstrap images inside of the collection
rcp45_2050s_Classified = makeBootstrapIC(rcp45_2050s_Mean_Soil,rcp45_2050s_Coll,seedChunkListToMap);

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(longWaitTime)
print('Moving on...')


You have jobs running!  2020-04-05 14:10:49
You have jobs running!  2020-04-05 14:20:50


In [ ]:
# Sum each of the image collections to get a finalized map
sumImageCollection(rcp45_2050s_Coll)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


In [ ]:
# rcp45_2050s_Coll
currentCollToDelete = rcp45_2050s_Coll

# Turn the coll string into an assetID and perform the deletion
assetIDToDelete = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToDelete
print(assetIDToDelete,'being deleted')

# Delete the image collection after summing the 
subprocess.run(bashCommandList_Delete+[assetIDToDelete])
while not all(x in subprocess.run(bashCommandList_Detect+[assetIDToDelete], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for the asset to delete...')
    time.sleep(5)
print('Collection is deleted!')


## rcp45_2070s

In [ ]:
# rcp45_2070s_Coll Image Collection creation
rcp45_2070s_Coll = 'rcp45_2070s_Coll';

# Turn the coll string into an assetID and perform the deletion
currentCollToCreate = rcp45_2070s_Coll
assetIDToCreate = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToCreate
print(assetIDToCreate,'being created...')

# Create the image collection before classifying each of the bootstrap images
subprocess.run(bashCommandList_CreateCollection+[assetIDToCreate])
while any(x in subprocess.run(bashCommandList_Detect+[assetIDToCreate], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for asset to be created...')
    time.sleep(normalWaitTime)
print('Asset created!')

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# Make the bootstrap images inside of the collection
rcp45_2070s_Classified = makeBootstrapIC(rcp45_2070s_Mean_Soil,rcp45_2070s_Coll,seedChunkListToMap);

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(longWaitTime)
print('Moving on...')


In [ ]:
# Sum each of the image collections to get a finalized map
sumImageCollection(rcp45_2070s_Coll)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


In [ ]:
# rcp45_2070s_Coll
currentCollToDelete = rcp45_2070s_Coll

# Turn the coll string into an assetID and perform the deletion
assetIDToDelete = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToDelete
print(assetIDToDelete,'being deleted')

# Delete the image collection after summing the 
subprocess.run(bashCommandList_Delete+[assetIDToDelete])
while not all(x in subprocess.run(bashCommandList_Detect+[assetIDToDelete], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for the asset to delete...')
    time.sleep(5)
print('Collection is deleted!')


## rcp45_2080s

In [ ]:
# rcp45_2080s_Coll Image Collection creation
rcp45_2080s_Coll = 'rcp45_2080s_Coll';

# Turn the coll string into an assetID and perform the deletion
currentCollToCreate = rcp45_2080s_Coll
assetIDToCreate = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToCreate
print(assetIDToCreate,'being created...')

# Create the image collection before classifying each of the bootstrap images
subprocess.run(bashCommandList_CreateCollection+[assetIDToCreate])
while any(x in subprocess.run(bashCommandList_Detect+[assetIDToCreate], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for asset to be created...')
    time.sleep(normalWaitTime)
print('Asset created!')

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# Make the bootstrap images inside of the collection
rcp45_2080s_Classified = makeBootstrapIC(rcp45_2080s_Mean_Soil,rcp45_2080s_Coll,seedChunkListToMap);

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(longWaitTime)
print('Moving on...')


In [ ]:
# Sum each of the image collections to get a finalized map
sumImageCollection(rcp45_2080s_Coll)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


In [ ]:
# rcp45_2080s_Coll
currentCollToDelete = rcp45_2080s_Coll

# Turn the coll string into an assetID and perform the deletion
assetIDToDelete = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToDelete
print(assetIDToDelete,'being deleted')

# Delete the image collection after summing the 
subprocess.run(bashCommandList_Delete+[assetIDToDelete])
while not all(x in subprocess.run(bashCommandList_Detect+[assetIDToDelete], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for the asset to delete...')
    time.sleep(5)
print('Collection is deleted!')


## rcp85_2030s

In [ ]:
# rcp85_2030s_Coll Image Collection creation
rcp85_2030s_Coll = 'rcp85_2030s_Coll';

# Turn the coll string into an assetID and perform the deletion
currentCollToCreate = rcp85_2030s_Coll
assetIDToCreate = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToCreate
print(assetIDToCreate,'being created...')

# Create the image collection before classifying each of the bootstrap images
subprocess.run(bashCommandList_CreateCollection+[assetIDToCreate])
while any(x in subprocess.run(bashCommandList_Detect+[assetIDToCreate], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for asset to be created...')
    time.sleep(normalWaitTime)
print('Asset created!')

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# Make the bootstrap images inside of the collection
rcp85_2030s_Classified = makeBootstrapIC(rcp85_2030s_Mean_Soil,rcp85_2030s_Coll,seedChunkListToMap);

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(longWaitTime)
print('Moving on...')


In [ ]:
# Sum each of the image collections to get a finalized map
sumImageCollection(rcp85_2030s_Coll)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


In [ ]:
# rcp85_2030s_Coll
currentCollToDelete = rcp85_2030s_Coll

# Turn the coll string into an assetID and perform the deletion
assetIDToDelete = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToDelete
print(assetIDToDelete,'being deleted')

# Delete the image collection after summing the 
subprocess.run(bashCommandList_Delete+[assetIDToDelete])
while not all(x in subprocess.run(bashCommandList_Detect+[assetIDToDelete], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for the asset to delete...')
    time.sleep(5)
print('Collection is deleted!')


## rcp85_2050s

In [ ]:
# rcp85_2050s_Coll Image Collection creation
rcp85_2050s_Coll = 'rcp85_2050s_Coll';

# Turn the coll string into an assetID and perform the deletion
currentCollToCreate = rcp85_2050s_Coll
assetIDToCreate = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToCreate
print(assetIDToCreate,'being created...')

# Create the image collection before classifying each of the bootstrap images
subprocess.run(bashCommandList_CreateCollection+[assetIDToCreate])
while any(x in subprocess.run(bashCommandList_Detect+[assetIDToCreate], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for asset to be created...')
    time.sleep(normalWaitTime)
print('Asset created!')

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# Make the bootstrap images inside of the collection
rcp85_2050s_Classified = makeBootstrapIC(rcp85_2050s_Mean_Soil,rcp85_2050s_Coll,seedChunkListToMap);

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# !! Break and wait
# Wait while all queued tasks finish
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(longWaitTime)
print('Moving on...')


In [ ]:
# Sum each of the image collections to get a finalized map
sumImageCollection(rcp85_2050s_Coll)


In [ ]:
# !! Break and wait
# Wait while all queued tasks finish
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


In [ ]:
# rcp85_2050s_Coll
currentCollToDelete = rcp85_2050s_Coll

# Turn the coll string into an assetID and perform the deletion
assetIDToDelete = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToDelete
print(assetIDToDelete,'being deleted')

# Delete the image collection after summing the 
subprocess.run(bashCommandList_Delete+[assetIDToDelete])
while not all(x in subprocess.run(bashCommandList_Detect+[assetIDToDelete], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for the asset to delete...')
    time.sleep(5)
print('Collection is deleted!')


## rcp85_2070s

In [ ]:
# rcp85_2070s_Coll Image Collection creation
rcp85_2070s_Coll = 'rcp85_2070s_Coll';

# Turn the coll string into an assetID and perform the deletion
currentCollToCreate = rcp85_2070s_Coll
assetIDToCreate = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToCreate
print(assetIDToCreate,'being created...')

# Create the image collection before classifying each of the bootstrap images
subprocess.run(bashCommandList_CreateCollection+[assetIDToCreate])
while any(x in subprocess.run(bashCommandList_Detect+[assetIDToCreate], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for asset to be created...')
    time.sleep(normalWaitTime)
print('Asset created!')

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# Turn the coll string into an assetID and perform the deletion
rcp85_2070s_Classified = makeBootstrapIC(rcp85_2070s_Mean_Soil,rcp85_2070s_Coll,seedChunkListToMap);

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(longWaitTime)
print('Moving on...')


In [ ]:
# Sum each of the image collections to get a finalized map
sumImageCollection(rcp85_2070s_Coll)


In [ ]:
# !! Break and wait
# Wait while all queued tasks finish
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


In [ ]:
# rcp85_2070s_Coll
currentCollToDelete = rcp85_2070s_Coll

# Turn the coll string into an assetID and perform the deletion
assetIDToDelete = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToDelete
print(assetIDToDelete,'being deleted')

# Delete the image collection after summing the 
subprocess.run(bashCommandList_Delete+[assetIDToDelete])
while not all(x in subprocess.run(bashCommandList_Detect+[assetIDToDelete], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for the asset to delete...')
    time.sleep(5)
print('Collection is deleted!')


## rcp85_2080s

In [ ]:
# rcp85_2080s_Coll Image Collection creation
rcp85_2080s_Coll = 'rcp85_2080s_Coll';

# Turn the coll string into an assetID and perform the deletion
currentCollToCreate = rcp85_2080s_Coll
assetIDToCreate = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToCreate
print(assetIDToCreate,'being created...')

# Create the image collection before classifying each of the bootstrap images
subprocess.run(bashCommandList_CreateCollection+[assetIDToCreate])
while any(x in subprocess.run(bashCommandList_Detect+[assetIDToCreate], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for asset to be created...')
    time.sleep(normalWaitTime)
print('Asset created!')

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# Create the image collection before classifying each of the bootstrap images
rcp85_2080s_Classified = makeBootstrapIC(rcp85_2080s_Mean_Soil,rcp85_2080s_Coll,seedChunkListToMap);

# Sleep to allow the server time to receive incoming requests
time.sleep(normalWaitTime/2)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(longWaitTime)
print('Moving on...')


In [ ]:
# Sum each of the image collections to get a finalized map
sumImageCollection(rcp85_2080s_Coll)


In [ ]:
# !! Break and wait
while any(x in str(ee.batch.Task.list()) for x in ['RUNNING','READY']):
    print('You have jobs running! ',datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
    time.sleep(normalWaitTime)
print('Moving on...')


In [ ]:
# rcp85_2080s_Coll
currentCollToDelete = rcp85_2080s_Coll

# Turn the coll string into an assetID and perform the deletion
assetIDToDelete = 'users/'+usernameFolderString+'/'+projectFolder+'/'+currentCollToDelete
print(assetIDToDelete,'being deleted')

# Delete the image collection after summing the 
subprocess.run(bashCommandList_Delete+[assetIDToDelete])
while not all(x in subprocess.run(bashCommandList_Detect+[assetIDToDelete], stdout=subprocess.PIPE).stdout.decode('utf-8') for x in stringsOfInterest):
    print('Waiting for the asset to delete...')
    time.sleep(5)
print('Collection is deleted!')
